#  Train a GPT-2 Text-Generating Model to write poetry. 
#### "Artificial intelligence versus Maya Angelou: Experimental evidence that people cannot differentiate AI-generated from human-written poetry" 

Article llink: https://www.sciencedirect.com/science/article/pii/S0747563220303034

by [Max Woolf](http://minimaxir.com) & edited by Luca Mossink

*Last updated: April 24th, 2020*

In the original Jupyter notebook you could retrain an advanced text generating neural network on any text dataset`gpt-2-simple`.

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read his [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!

This notebook was run on Google Colab.
To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes(File -> Save a Copy in Drive).
2. Make sure your hardware accelerator is set to GPU. If you want to train the full model, it is advised to use a more powerful GPU. For the experiment, Köbis & Mossink used a Tesla V100.
3. Run the cells below:


In [ ]:
import sys 
!{sys.executable} -m pip install tensorflow==1.15.0

In [ ]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
#from google.colab import files

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

You may run this notebook Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Mon Apr 13 16:14:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    48W / 300W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   34C    P0    46W / 300W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

## Downloading GPT-2

First, you need to download the GPT-2 model you want to use. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="774M")

In [ ]:
gpt2.download_gpt2(model_name="1558M")

Fetching checkpoint: 1.05Mit [00:00, 531Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 96.4Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 1.16Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 6.23Git [01:24, 73.4Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 733Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 116Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 119Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "training_data_poem12.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='1558M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=2000,
              save_every=1000
              )

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='1558M',
              steps=2500,
              restore_from='latest',
              overwrite='True',
              run_name='run1',
              print_every=10,
              sample_every=2500,
              save_every=2500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint checkpoint/run1/model-1281
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1281


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:03<00:00,  3.44s/it]


dataset has 575706 tokens
Training...
Saving checkpoint/run1/model-1281
[1290 | 378.20] loss=1.36 avg=1.36
[1300 | 744.07] loss=2.67 avg=2.02
[1310 | 1111.92] loss=1.99 avg=2.01
[1320 | 1477.36] loss=1.28 avg=1.82
[1330 | 1844.53] loss=0.58 avg=1.57
[1340 | 2212.34] loss=4.14 avg=2.01
[1350 | 2582.46] loss=0.85 avg=1.84
[1360 | 2951.68] loss=0.77 avg=1.70
[1370 | 3322.51] loss=0.41 avg=1.55
[1380 | 3693.94] loss=0.46 avg=1.44
[1390 | 4067.96] loss=3.14 avg=1.60
[1400 | 4444.90] loss=0.15 avg=1.47
[1410 | 4821.79] loss=1.09 avg=1.44
[1420 | 5196.94] loss=0.91 avg=1.40
[1430 | 5571.90] loss=0.78 avg=1.36
[1440 | 5949.34] loss=0.18 avg=1.28
[1450 | 6327.16] loss=0.62 avg=1.24
[1460 | 6702.32] loss=0.83 avg=1.21
[1470 | 7079.60] loss=3.56 avg=1.35
[1480 | 7454.59] loss=0.40 avg=1.29
[1490 | 7830.41] loss=0.41 avg=1.25
[1500 | 8208.09] loss=0.30 avg=1.20
[1510 | 8584.19] loss=0.33 avg=1.16
[1520 | 8959.63] loss=1.75 avg=1.19
[1530 | 9336.34] loss=2.75 avg=1.26
[1540 | 9711.01] loss=0.49 avg

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1823
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1823


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=75,
              temperature=0.9,
              top_k=40,
              top_p=0.9,
              prefix="They burnt a corpse upon the sand-- The light shone out afar;",
              nsamples=20,
              batch_size=10
              )

They burnt a corpse upon the sand-- The light shone out afar;
                                                                          
They burnt a corpse upon the sand-- The light shone out afar;
                                                                          
They burnt a corpse upon the sand-- The light shone out afar;
                                                                          
They burnt a corpse upon the sand-- The light shone out afar;
And down a green plain, leaping, laughing, squealed:
"Ahoy! who is this laughing thing?
  I'll give my stags, my hills and dales,
  My stormcocks and my nightingales.

"And I'll chew up your silks,--you're dead,--
  You'll
They burnt a corpse upon the sand-- The light shone out afar;
                                                                          
They burnt a corpse upon the sand-- The light shone out afar;
                                                                          
They burnt a corpse upon the s

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=100,
                      temperature=0.9,
                      top_k=40,
                      nsamples=20,
                      batch_size=10
                      )

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=75,
                      temperature=0.85,
                      prefix="Everyday, I think about dying.",
                      top_k=40,
                      nsamples=20,
                      batch_size=10
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "1558M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 508Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 132Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 736Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 6.23Git [02:07, 48.9Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 556Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 216Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 238Mit/s]                                                       


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

Loading pretrained model models/1558M/model.ckpt
INFO:tensorflow:Restoring parameters from models/1558M/model.ckpt


In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is to take risks whenever possible",
              length=500,
              temperature=0.9,
              top_p=0.9,
              nsamples=10,
              batch_size=5
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The secret of life is to take risks whenever possible," I had told him, "and to get comfortable with small doses of pleasure. But one should never allow anything to become a substitute for a deep conviction in what is good."

Although he had tried to please me, Dr. Mössbauer could not find me in his list. He had gone on to a similar list of my friends and acquaintances. I turned away.

In the second room, where he had also made his initial impression of me, he stopped short. I was a little surprised. He made his intentions plain enough. He was interested in what I had to say, but didn't know how to express it.

I was prepared to take him under my wing. It would be only a short visit, and we could get to know one another, learn about each other's lives.

But I was tired of the whole situation. He had shown no curiosity about my ideas, and the moment he had realized I didn't know them well enoug

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.